In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import time
import xgboost as xgb
import os, time
from tqdm import tqdm
import pyarrow.parquet as pq
import pyarrow as pa

import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

import sklearn
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import tqdm, re, pickle, gc, os, time, random

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from collections import defaultdict

In [2]:
def predict_with_trained_model_v2(model_name, feas_file_path, oot_append, predict_data_batch = 5): 
    '''
    model_name: str of the model's name. 
    oot_append: dataframe.
    predict_data_batch: how many batches the oot_append will be splitted. 
    '''
    model = xgb.Booster({'nthread': 4}) ## ns: not strict labeled
    model.load_model(f'trained_models/{model_name}')
    
    with open(feas_file_path, "r") as f:
        feas_cols_v2 = [i.strip() for i in f.readlines()]
    
    print(f"How many features are used? ", len(feas_cols_v2))
    
    n_batches = predict_data_batch
    nrow_interval = oot_append.shape[0] // n_batches - 5 ## 把原来的数据分成5份，然后逐份来做预测。否则实在太久。

    ns_prob_col = None

    for i in tqdm.tqdm(range(n_batches + 1)):
        if i == n_batches:
            partial_data_to_be_predict = oot_append.loc[i*nrow_interval:, feas_cols_v2]
        else:
            partial_data_to_be_predict = oot_append.loc[i*nrow_interval: (i+1)*nrow_interval - 1, feas_cols_v2] ## 用loc来切片，“行”这一维度，竟然是上下限都保留！！
        ##################### 
        if ns_prob_col is None:
            ns_prob_col = model.predict(xgb.DMatrix(partial_data_to_be_predict[feas_cols_v2].values))
        else:
            ns_prob_col = np.hstack(
                [
                    ns_prob_col, 
                    model.predict(xgb.DMatrix(partial_data_to_be_predict[feas_cols_v2].values))
                ]
            )
    
    return ns_prob_col

def loadTheData_doThePrediction_v2(model_name, feas_file_path, df, score_df_path, predict_data_batch = 5, score_col_name = "score", id_col_name = "trace_id"):
    scores = predict_with_trained_model_v2(
        model_name, 
        feas_file_path, 
        df, 
        predict_data_batch
    )
    pd.DataFrame(
        {
            id_col_name: df[id_col_name],
            score_col_name: scores,
        }
    ).to_csv(score_df_path, index=False)
    print(f"{model_name} prediction done!")
    
def get_all_feas(
    feas_file_name,
    feas_data_fname,
    feas_file_dir = f"/data/private/public_data/cpu1/minkexiu/thirdParty_feas_list/",
    feas_data_dir = f"/data/private/public_data/cpu1/minkexiu/parquet_data/",
    left_on = "trace_id", 
    right_on = "traceid",
):
    '''
    把前面的代码合集一下，放到这里。
    '''
    feas_file_path = os.path.join(feas_file_dir, feas_file_name)
    with open(feas_file_path, "r") as f:
        feas_cols_v2_upper = [i.strip() for i in f.readlines()]
    feas_cols_v2 = [_.lower() for _ in feas_cols_v2_upper]
    feas_data_path = os.path.join(feas_data_dir, feas_data_fname)
    df = pd.read_parquet(feas_data_path)
    
    print("here i am")
    label_data = pd.read_csv(
        # os.path.join(feas_data_dir, f"trace_ids-target_1d.csv")
        f"/data/private/public_data/cpu1/minkexiu/trace_id-info-202201_202204.csv"
    )
    print("merging the data...")
    all_feas = label_data.merge(df, how='left', left_on=left_on, right_on = right_on)
    
    del df, label_data
    gc.collect()
    
    return all_feas, feas_cols_v2_upper, feas_cols_v2

In [3]:
def get_tmpList(nm, actual_nFeas, itv, n_iter=11):
    l = [nm.rsplit("-", 1)[-1] + "-origin"]
    for j in range(n_iter):
        i = actual_nFeas-j*itv
        l.append(i)
    return l

In [4]:
nm = "pre1"
target_label = "isDefault"
userid_col = "id" ## loan_account_id
traceid_col = "issueDate" ## trace_id
n_fold = 5
NEED_TO_PREPARE_DATA = False ## 用这个标记一下，是否需要重新准备数据
NEED_TO_TRAIN = False ## 用这个标记一下，是否需要重新准备数据

In [5]:
with open(f"preprocessedDataset/{nm}.txt", "r") as f:
    feas_cols_v2 = [i.strip() for i in f.readlines()]
len(feas_cols_v2)

49

# 得到评分

In [11]:
if not os.path.exists("pred_scores/"):
    os.makedirs("pred_scores/")

for i in range(n_fold):
    print("*"*30, f"getting prediction stores id residual is {i}......")
    all_feas_pred = pq.read_table(
        f"preprocessedDataset/{nm}-IDResd_{i}.parquet",
        columns=feas_cols_v2 + [traceid_col]
    ).to_pandas()
    
    tmp_name = f"{nm}-IDResd_{i}"
    loadTheData_doThePrediction_v2(
        model_name = f"{tmp_name}",
        feas_file_path = f"preprocessedDataset/{nm}.txt",
        df = all_feas_pred, 
        score_df_path = f"pred_scores/{tmp_name}.csv", 
        predict_data_batch = 20,
        score_col_name=f"{nm}--score",
        id_col_name = traceid_col,
    )

****************************** getting prediction stores id residual is 0......
How many features are used?  49


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 157.41it/s]


pre1-IDResd_0 prediction done!
****************************** getting prediction stores id residual is 1......
How many features are used?  49


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 143.68it/s]


pre1-IDResd_1 prediction done!
****************************** getting prediction stores id residual is 2......
How many features are used?  49


...

pre1-IDResd_2 prediction done!
****************************** getting prediction stores id residual is 3......
How many features are used?  49


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 186.26it/s]


pre1-IDResd_3 prediction done!
****************************** getting prediction stores id residual is 4......
How many features are used?  49


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 221.72it/s]


pre1-IDResd_4 prediction done!


# 把评分拼起来

In [12]:
scores = None
for i in range(n_fold):
    tmp_name = f"{nm}-IDResd_{i}"
    print(tmp_name)
    if scores is None:
        scores = pd.read_csv(f"pred_scores/{tmp_name}.csv")
    else:
        scores = pd.concat([scores, pd.read_csv(f"pred_scores/{tmp_name}.csv")])
scores.to_csv(f"pred_scores/{nm}-maidianScore.csv", index=False)
scores.shape

pre1-IDResd_0
pre1-IDResd_1
pre1-IDResd_2
pre1-IDResd_3
pre1-IDResd_4


(800000, 2)